In [3]:
# Test tts_util.py
from tts_util import OpenAiTTSProvider

ttsProvider = OpenAiTTSProvider()
print(ttsProvider.generate_speech('echo', 'Calm and soothing', 'Hello, world! How is it going. I am Abhilash. This is a test audio. I am an entrepreneure. I like to build cool things. You mother fucker', './audio_outputs', 1, 'test'))

INFO:tts_util:Generating audio for chunk 1_test, attempt 1
INFO:tts_util:Successfully generated audio: ./audio_outputs/001_test.wav


./audio_outputs/001_test.wav


In [4]:
# Test chunk creation util
from text_chunking_util import SimpleTextChunkCreator
chunkCreator = SimpleTextChunkCreator()
text = """Hey there. I am testing chunking logic.It should chunk based on new line characters."""
chunks = chunkCreator.convertTextToChunks(text, "\n")
print(chunks)

['Hey there. I am testing chunking logic.It should chunk based on new line characters.']


In [5]:
import uuid
from db_utils import DatabaseManager
from text_chunking_util import SimpleTextChunkCreator
from models.db_models import AudioChunk


def saveChunksForJob(db_manager: DatabaseManager, job_id: str):
    """
    Create audio chunks from job's firstDraft and save to database
    
    Args:
        db_manager: Database manager instance
        job_id: ID of the job to process
    """
    # Fetch job
    job = db_manager.get_job(job_id)
    if not job:
        raise ValueError(f"Job with ID {job_id} not found")
    
    # Get firstDraft text
    first_draft = job.firstDraft
    if not first_draft:
        raise ValueError(f"Job {job_id} has no firstDraft content")
    
    # Create chunks using SimpleTextChunkCreator
    chunker = SimpleTextChunkCreator()
    text_chunks = chunker.convertTextToChunks(first_draft, '\n')
    
    # Convert to AudioChunk objects with unique IDs
    audio_chunks = []
    for index, chunk_text in enumerate(text_chunks):
        chunk_id = str(uuid.uuid4())[:8]  # Short unique ID
        
        audio_chunk = AudioChunk(
            chunkId=chunk_id,
            text=chunk_text,
            outputAudioFilePath=""  # Will be populated during audio generation
        )
        audio_chunks.append(audio_chunk)
    
    # Update job with audio chunks
    job.audioChunks = audio_chunks
    db_manager.update_job_field(job_id, job)
    
    print(f"✓ Created {len(audio_chunks)} audio chunks for job {job_id}")
    return audio_chunks

In [6]:
import os
from dotenv import load_dotenv
from db_utils import DatabaseManager

if __name__ == "__main__":
    load_dotenv()
    
    # Initialize database manager
    db_manager = DatabaseManager()
    
    # Set the job ID (replace with actual job ID from story generation)
    job_id = "68d19cc430f1f23c880f32a4"
    
    # Create and save audio chunks for the job
    audio_chunks = saveChunksForJob(db_manager, job_id)
    
    print(f"Processed {len(audio_chunks)} chunks:")
    for i, chunk in enumerate(audio_chunks):
        print(f"  Chunk {i+1}: ID={chunk.chunkId}, Length={len(chunk.text)} chars")
    
    # Close DB connection when done
    db_manager.close()

✓ Created 199 audio chunks for job 68d19cc430f1f23c880f32a4
Processed 199 chunks:
  Chunk 1: ID=0e618d69, Length=31 chars
  Chunk 2: ID=5809c896, Length=1 chars
  Chunk 3: ID=fb4eb563, Length=502 chars
  Chunk 4: ID=fbf5f762, Length=465 chars
  Chunk 5: ID=ae225a4e, Length=492 chars
  Chunk 6: ID=e84ab013, Length=322 chars
  Chunk 7: ID=4324e4ab, Length=367 chars
  Chunk 8: ID=38dec01b, Length=331 chars
  Chunk 9: ID=f2bdb690, Length=472 chars
  Chunk 10: ID=ec7220ca, Length=367 chars
  Chunk 11: ID=1f348f8b, Length=158 chars
  Chunk 12: ID=41f2c2a7, Length=57 chars
  Chunk 13: ID=2e6ebe72, Length=347 chars
  Chunk 14: ID=a6fa0625, Length=660 chars
  Chunk 15: ID=efd40415, Length=489 chars
  Chunk 16: ID=cc4cec4f, Length=265 chars
  Chunk 17: ID=e83ae4e1, Length=118 chars
  Chunk 18: ID=67fbecc8, Length=373 chars
  Chunk 19: ID=f0a1f9b2, Length=212 chars
  Chunk 20: ID=48141be7, Length=70 chars
  Chunk 21: ID=bd48bc18, Length=242 chars
  Chunk 22: ID=a3335d60, Length=274 chars
  Chunk 